# Neural Networks

This Jupyter Notebook uses neural networks to TBD.

<!--

## Approach
* Evaluations of classification performed were performed using 2 sample datasets:
    * sklearn's Iris sample dataset, consisting of 150 sample data points for 3 varieties of iris flowers
    * voice dataset, containing 3168 sample data points of male and female speakers

* Hyper-parameter tuning was performed using a GridSearch estimator for each classification model:
    
* Parameters providing the best scores and associated scores were plotted
    
## Results
* Refer to the Figures and Table of results provided below
* Hyperparameter tuning provided by sklearn GridSearchCV provided a convenient method of evaluating many classification model parameter sets in an efficient way.  The blue dots represent the optimal parameter sets selected for each Dataset/Classification Model, and the grey dots represent suboptimal parameter sets.
* For the Iris dataset, the Support Vector Machine SVC model provided the  combination of best score of 0.964 and Mean Fit Time of 0.200 ms, with Random Forest and Decision Tree classifiers providing close performance.
* For the Voice dataset, the SVC classifer also had the hightest Best Score at 0.980, which was slightly above the performance of the Random Forest classifier.  With SVC, the fit time was singificantly higher for the Voice dataset (40.6 ms) than for the Iris dataset (0.2 ms), which is expected given the larger number of features associated with the Voice dataset (20) vs. the Iris dataset (4)
* It's interesting to note that the K-Nearest Neighbors classifier, while performing lower amongst these models, operated with fast Mean Fit Time (3.2 ms) for Voice, which might make using KNN a good choice vs. other more calculation-intensive options in cases where lower fit time is more important than optimum accuracy.

-->

| Figure: Neural Network Performance: Best Score vs. Mean Fit Time (ms) | 
| :----------: |
| ![Figure: Neural Network Performance: Best Score vs. Mean Fit Time (ms) is Loading...](docs/Figure-Neural_Network_Performance-A.png "Figure: Neural Network Performance: Best Score vs. Mean Fit Time (ms)") |

<!--

| Figure: Tuned Classifier Performance: Best Score vs. F1 Score - All Datasets/Classifiers | Figure: Tuned Classifier Performance: Precision vs. Recall - All Datasets/Classifiers |
| :----------: | :----------: |
| ![Figure: Tuned Classifier Performance: Best Score vs. F1 Score - All Datasets/Classifiers is Loading...](docs/Figure-Hyper_Parameter_Tuning-BestScore_vs_F1-Combined.png "Figure: uned Classifier Performance: Best Score vs. F1 Score - All Datasets/Classifiers") | ![Figure: Tuned Classifier Performance: Precision vs. Recall - All Datasets/Classifiers is Loading...](docs/Figure-Hyper_Parameter_Tuning-Precision_vs_Recall-Combined.png "Figure: Tuned Classifier Performance: Precision vs. Recall - All Datasets/Classifiers") |

| Figure: Tuned Classifier Performance - Subplots |
| :----------: |
| ![Figure: Tuned Classifier Performance - Subplots is Loading...](docs/Figure-Hyper_Parameter_Tuning-Subplots.png "Figure: Tuned Classifier Performance - Subplots") |

| Table: Tuned Classifier Performance |
| :----------: |
| ![Table: Tuned Classifier Performance is Loading...](docs/Table-Hyper_Parameter_Tuning.png "Table: Tuned Classifier Performance") |

-->

# Dependencies

In [18]:
%matplotlib inline
# %matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D    # Support 3D graphing

import numpy as np
import pandas as pd
from pandas.plotting import table
import math

from pprint import pprint

# Visualization
import graphviz
import pydotplus
from IPython.display import Image

# Machine Learning - Linear Models - Regression
from sklearn.linear_model import LinearRegression  # TBD
from sklearn.linear_model import Lasso             # TBD
from sklearn.linear_model import Ridge             # TBD
from sklearn.linear_model import ElasticNet        # TBD

# Machine Learning - Linear Models - Classification
from sklearn.linear_model import LogisticRegression   # Logistic Regression Classifier

# Machine Learning - Decision Trees and Random Forests - Classification
from sklearn import tree                             # Decision Tree Classifier
from sklearn.ensemble import RandomForestClassifier  # Random Forest Classifier

# Machine Learning - Support Vector Machines - Classification
from sklearn import svm                              # Support Vector Machine Classifier

# Machine Learning - K-Nearest Neighbors - Classification
from sklearn.neighbors import KNeighborsClassifier   # K-Nearest Neighbors (KNN)

# Machine Learning - GridSearch for hyper-parameter tuning
from sklearn.model_selection import GridSearchCV


# Machine Learning - Data Preparation and Pre-Processing
from sklearn.model_selection import train_test_split # Split data into training and testing samples
from sklearn.model_selection import cross_val_score  # Score a model using k-fold or other cross validation

from sklearn.preprocessing import OneHotEncoder   # Convert categorical integer features (X) to One-Hot encoded values
from sklearn.preprocessing import LabelEncoder    # Convert categorical labeled values to categorical integer values
from sklearn.preprocessing import LabelBinarizer  # Convert categorical labeled values to Binary encoded values

from sklearn.preprocessing import StandardScaler  # Scale numerical features to standard normal distribution
from sklearn.preprocessing import MinMaxScaler    # Scale numerical values based upon mix/max values

# Machine Learning - Neural Networks - Models
from keras.models import Sequential               # Sequential model serving as foundation for neural network
from keras.layers import Dense                    # Nodes for specifying input, hidden, and output layers

# Machine Learning - Neural Networks - Encoding
from keras.utils import to_categorical            # One-Hot Encoder provided through Keras

# Machine Learning - Other Models-related Tools
from keras.utils import plot_model                # Plot a neural network model
from keras.models import load_model               # Load a saved machine learning model

# Machine Learning - Quantify Model Performance
from sklearn.metrics import mean_squared_error    # Mean Squared Error (MSE) metric
from sklearn.metrics import r2_score              # R-squared (Coefficient of Determination) metric
from sklearn.metrics import confusion_matrix      # Generate a confusion matrix (actual vs. predicted counts)
from sklearn.metrics import classification_report # Calculate metrics for prediction performance
from sklearn.metrics import precision_score       # Calculate the precision: Tp / (Tp + Fp) => Ability to avoid false negatives
from sklearn.metrics import recall_score          # Calculate the recall: Tp / (Tp + Fn) => Ability to find all positive samples
from sklearn.metrics import f1_score              # Calculate the F1 score: 2*(precision*recall)/(precision+recall)

# Machine Learning - Dataset Generation
from sklearn.datasets import make_regression     # Generate linear data
from sklearn.datasets import make_s_curve        # Generate nonlinear data
from sklearn.datasets import make_blobs          # Generate blobs for classification
from sklearn.datasets import make_circles        # Generate circles for classification
from sklearn.datasets import load_iris           # Sample multi-class dataset for classification
from sklearn.datasets import make_classification # Generate 

# Iris Dataset

In [2]:
# Import the Iris sample dataset as a dictionary
iris = load_iris()

print( f"Iris {iris.keys()}" )
print( f"Feature Names: {iris.feature_names}" )
print( f"Target Names: {iris.target_names}" )

Iris dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])
Feature Names: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Target Names: ['setosa' 'versicolor' 'virginica']


In [3]:
# Assign data and target variables
X = iris.data
y = iris.target
print( X.shape, y.shape )

(150, 4) (150,)


In [4]:
# Note, in this case, Label Encoding of the target/output is not needed
# because the target is already encoded as integers 0 through 2 inclusive,
# corresponding to the target names in iris.target_names
pd.DataFrame( data={ 'Target': list(set(iris.target)), 'Target Name': iris.target_names }).set_index(keys='Target')

,Target Name
Target,
0,setosa
1,versicolor
2,virginica


In [5]:
# Apply One-Hot Encoding of the label-encoded target values
y_encoded = to_categorical(y)
# y_encoded

In [6]:
# Split the data using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, random_state=1, stratify=y_encoded)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(112, 4) (38, 4) (112, 3) (38, 3)


In [7]:
# # Reshape the outputs to provide a N by 1 vector (vs. array of size N) -- Not with y being encoded
# y_train = y_train.reshape(-1,1)
# y_test = y_test.reshape(-1,1)
# print(y_train.shape, y_test.shape)

In [8]:
# Use the training data to create a scaler to standard normal distributions for each numerical feature and output
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)

In [9]:
# Scale the training and test data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

### Define the Neural Network model for multi-class, single-label classification

In [10]:
# Create a sequential model
model = Sequential()

In [11]:
# Add layer:
# Number of Inputs: 4
# Number of Hidden Nodes: 4
model.add(Dense(units=4, activation='relu', input_dim=4))

# Add output layer:
# Number of Classes: 3 = # of output nodes (units) needed
model.add(Dense(units=3, activation='softmax'))

# Provide a summary of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 15        
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________


### Compile the Model

In [12]:
# Compile the model
# Optimizer: Adam
# Loss Function: Categorical Cross-Entropy for categorical data (Mean Squared Error for regression)
# Metrics: Accuracy
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### Train the Model

In [13]:
%%timeit -n1 -r1
# Fit (train) the model
hist = model.fit(
    X_train_scaled,
    y_train_scaled,
    epochs=1000,
    shuffle=True,
    verbose=2
)

Epoch 1/1000
 - 0s - loss: -4.4980e-01 - acc: 0.5357
Epoch 2/1000
 - 0s - loss: -4.6972e-01 - acc: 0.5446
Epoch 3/1000
 - 0s - loss: -4.9072e-01 - acc: 0.5179
Epoch 4/1000
 - 0s - loss: -5.1362e-01 - acc: 0.5179
Epoch 5/1000
 - 0s - loss: -5.3585e-01 - acc: 0.5179
Epoch 6/1000
 - 0s - loss: -5.5806e-01 - acc: 0.5357
Epoch 7/1000
 - 0s - loss: -5.8243e-01 - acc: 0.5893
Epoch 8/1000
 - 0s - loss: -6.0645e-01 - acc: 0.5893
Epoch 9/1000
 - 0s - loss: -6.3074e-01 - acc: 0.5804
Epoch 10/1000
 - 0s - loss: -6.5526e-01 - acc: 0.5536
Epoch 11/1000
 - 0s - loss: -6.8171e-01 - acc: 0.5446
Epoch 12/1000
 - 0s - loss: -7.0833e-01 - acc: 0.5357
Epoch 13/1000
 - 0s - loss: -7.3491e-01 - acc: 0.5268
Epoch 14/1000
 - 0s - loss: -7.6457e-01 - acc: 0.5268
Epoch 15/1000
 - 0s - loss: -7.9260e-01 - acc: 0.5357
Epoch 16/1000
 - 0s - loss: -8.2293e-01 - acc: 0.5357
Epoch 17/1000
 - 0s - loss: -8.5188e-01 - acc: 0.5357
Epoch 18/1000
 - 0s - loss: -8.8274e-01 - acc: 0.5446
Epoch 19/1000
 - 0s - loss: -9.1217e-

Epoch 152/1000
 - 0s - loss: -1.0619e+01 - acc: 0.8214
Epoch 153/1000
 - 0s - loss: -1.0716e+01 - acc: 0.8214
Epoch 154/1000
 - 0s - loss: -1.0807e+01 - acc: 0.8214
Epoch 155/1000
 - 0s - loss: -1.0899e+01 - acc: 0.8214
Epoch 156/1000
 - 0s - loss: -1.0994e+01 - acc: 0.8214
Epoch 157/1000
 - 0s - loss: -1.1087e+01 - acc: 0.8214
Epoch 158/1000
 - 0s - loss: -1.1180e+01 - acc: 0.8214
Epoch 159/1000
 - 0s - loss: -1.1273e+01 - acc: 0.8214
Epoch 160/1000
 - 0s - loss: -1.1368e+01 - acc: 0.8214
Epoch 161/1000
 - 0s - loss: -1.1462e+01 - acc: 0.8214
Epoch 162/1000
 - 0s - loss: -1.1555e+01 - acc: 0.8214
Epoch 163/1000
 - 0s - loss: -1.1648e+01 - acc: 0.8214
Epoch 164/1000
 - 0s - loss: -1.1742e+01 - acc: 0.8214
Epoch 165/1000
 - 0s - loss: -1.1830e+01 - acc: 0.8214
Epoch 166/1000
 - 0s - loss: -1.1917e+01 - acc: 0.8214
Epoch 167/1000
 - 0s - loss: -1.2007e+01 - acc: 0.8214
Epoch 168/1000
 - 0s - loss: -1.2097e+01 - acc: 0.8214
Epoch 169/1000
 - 0s - loss: -1.2182e+01 - acc: 0.8214
Epoch 170/

Epoch 301/1000
 - 0s - loss: -1.7316e+01 - acc: 0.8661
Epoch 302/1000
 - 0s - loss: -1.7330e+01 - acc: 0.8661
Epoch 303/1000
 - 0s - loss: -1.7341e+01 - acc: 0.8661
Epoch 304/1000
 - 0s - loss: -1.7353e+01 - acc: 0.8661
Epoch 305/1000
 - 0s - loss: -1.7366e+01 - acc: 0.8661
Epoch 306/1000
 - 0s - loss: -1.7377e+01 - acc: 0.8661
Epoch 307/1000
 - 0s - loss: -1.7389e+01 - acc: 0.8661
Epoch 308/1000
 - 0s - loss: -1.7400e+01 - acc: 0.8661
Epoch 309/1000
 - 0s - loss: -1.7410e+01 - acc: 0.8750
Epoch 310/1000
 - 0s - loss: -1.7421e+01 - acc: 0.8750
Epoch 311/1000
 - 0s - loss: -1.7432e+01 - acc: 0.8750
Epoch 312/1000
 - 0s - loss: -1.7442e+01 - acc: 0.8750
Epoch 313/1000
 - 0s - loss: -1.7451e+01 - acc: 0.8750
Epoch 314/1000
 - 0s - loss: -1.7461e+01 - acc: 0.8750
Epoch 315/1000
 - 0s - loss: -1.7471e+01 - acc: 0.8750
Epoch 316/1000
 - 0s - loss: -1.7481e+01 - acc: 0.8839
Epoch 317/1000
 - 0s - loss: -1.7490e+01 - acc: 0.8839
Epoch 318/1000
 - 0s - loss: -1.7501e+01 - acc: 0.8839
Epoch 319/

Epoch 450/1000
 - 0s - loss: -1.8241e+01 - acc: 0.8929
Epoch 451/1000
 - 0s - loss: -1.8246e+01 - acc: 0.8929
Epoch 452/1000
 - 0s - loss: -1.8250e+01 - acc: 0.8929
Epoch 453/1000
 - 0s - loss: -1.8255e+01 - acc: 0.8929
Epoch 454/1000
 - 0s - loss: -1.8259e+01 - acc: 0.8929
Epoch 455/1000
 - 0s - loss: -1.8264e+01 - acc: 0.8929
Epoch 456/1000
 - 0s - loss: -1.8268e+01 - acc: 0.8929
Epoch 457/1000
 - 0s - loss: -1.8273e+01 - acc: 0.8929
Epoch 458/1000
 - 0s - loss: -1.8277e+01 - acc: 0.8929
Epoch 459/1000
 - 0s - loss: -1.8281e+01 - acc: 0.8929
Epoch 460/1000
 - 0s - loss: -1.8286e+01 - acc: 0.8929
Epoch 461/1000
 - 0s - loss: -1.8290e+01 - acc: 0.8929
Epoch 462/1000
 - 0s - loss: -1.8294e+01 - acc: 0.8929
Epoch 463/1000
 - 0s - loss: -1.8299e+01 - acc: 0.8929
Epoch 464/1000
 - 0s - loss: -1.8304e+01 - acc: 0.8929
Epoch 465/1000
 - 0s - loss: -1.8309e+01 - acc: 0.8839
Epoch 466/1000
 - 0s - loss: -1.8313e+01 - acc: 0.8839
Epoch 467/1000
 - 0s - loss: -1.8317e+01 - acc: 0.8839
Epoch 468/

Epoch 599/1000
 - 0s - loss: -1.8882e+01 - acc: 0.8929
Epoch 600/1000
 - 0s - loss: -1.8886e+01 - acc: 0.8929
Epoch 601/1000
 - 0s - loss: -1.8890e+01 - acc: 0.8929
Epoch 602/1000
 - 0s - loss: -1.8894e+01 - acc: 0.8929
Epoch 603/1000
 - 0s - loss: -1.8898e+01 - acc: 0.8929
Epoch 604/1000
 - 0s - loss: -1.8903e+01 - acc: 0.8929
Epoch 605/1000
 - 0s - loss: -1.8907e+01 - acc: 0.8929
Epoch 606/1000
 - 0s - loss: -1.8911e+01 - acc: 0.8929
Epoch 607/1000
 - 0s - loss: -1.8915e+01 - acc: 0.9018
Epoch 608/1000
 - 0s - loss: -1.8920e+01 - acc: 0.9018
Epoch 609/1000
 - 0s - loss: -1.8924e+01 - acc: 0.9018
Epoch 610/1000
 - 0s - loss: -1.8928e+01 - acc: 0.9018
Epoch 611/1000
 - 0s - loss: -1.8933e+01 - acc: 0.9018
Epoch 612/1000
 - 0s - loss: -1.8937e+01 - acc: 0.9018
Epoch 613/1000
 - 0s - loss: -1.8941e+01 - acc: 0.9018
Epoch 614/1000
 - 0s - loss: -1.8946e+01 - acc: 0.9018
Epoch 615/1000
 - 0s - loss: -1.8950e+01 - acc: 0.9018
Epoch 616/1000
 - 0s - loss: -1.8954e+01 - acc: 0.9018
Epoch 617/

Epoch 748/1000
 - 0s - loss: -1.9546e+01 - acc: 0.9107
Epoch 749/1000
 - 0s - loss: -1.9550e+01 - acc: 0.9107
Epoch 750/1000
 - 0s - loss: -1.9554e+01 - acc: 0.9107
Epoch 751/1000
 - 0s - loss: -1.9559e+01 - acc: 0.9107
Epoch 752/1000
 - 0s - loss: -1.9564e+01 - acc: 0.9107
Epoch 753/1000
 - 0s - loss: -1.9568e+01 - acc: 0.9107
Epoch 754/1000
 - 0s - loss: -1.9573e+01 - acc: 0.9107
Epoch 755/1000
 - 0s - loss: -1.9577e+01 - acc: 0.9107
Epoch 756/1000
 - 0s - loss: -1.9582e+01 - acc: 0.9107
Epoch 757/1000
 - 0s - loss: -1.9585e+01 - acc: 0.9107
Epoch 758/1000
 - 0s - loss: -1.9590e+01 - acc: 0.9107
Epoch 759/1000
 - 0s - loss: -1.9594e+01 - acc: 0.9107
Epoch 760/1000
 - 0s - loss: -1.9599e+01 - acc: 0.9107
Epoch 761/1000
 - 0s - loss: -1.9603e+01 - acc: 0.9107
Epoch 762/1000
 - 0s - loss: -1.9607e+01 - acc: 0.9107
Epoch 763/1000
 - 0s - loss: -1.9611e+01 - acc: 0.9196
Epoch 764/1000
 - 0s - loss: -1.9615e+01 - acc: 0.9196
Epoch 765/1000
 - 0s - loss: -1.9619e+01 - acc: 0.9196
Epoch 766/

Epoch 897/1000
 - 0s - loss: -2.0136e+01 - acc: 0.9375
Epoch 898/1000
 - 0s - loss: -2.0139e+01 - acc: 0.9375
Epoch 899/1000
 - 0s - loss: -2.0143e+01 - acc: 0.9375
Epoch 900/1000
 - 0s - loss: -2.0147e+01 - acc: 0.9375
Epoch 901/1000
 - 0s - loss: -2.0150e+01 - acc: 0.9375
Epoch 902/1000
 - 0s - loss: -2.0153e+01 - acc: 0.9464
Epoch 903/1000
 - 0s - loss: -2.0157e+01 - acc: 0.9464
Epoch 904/1000
 - 0s - loss: -2.0161e+01 - acc: 0.9464
Epoch 905/1000
 - 0s - loss: -2.0165e+01 - acc: 0.9464
Epoch 906/1000
 - 0s - loss: -2.0167e+01 - acc: 0.9464
Epoch 907/1000
 - 0s - loss: -2.0172e+01 - acc: 0.9464
Epoch 908/1000
 - 0s - loss: -2.0176e+01 - acc: 0.9464
Epoch 909/1000
 - 0s - loss: -2.0180e+01 - acc: 0.9464
Epoch 910/1000
 - 0s - loss: -2.0183e+01 - acc: 0.9464
Epoch 911/1000
 - 0s - loss: -2.0186e+01 - acc: 0.9464
Epoch 912/1000
 - 0s - loss: -2.0190e+01 - acc: 0.9464
Epoch 913/1000
 - 0s - loss: -2.0194e+01 - acc: 0.9464
Epoch 914/1000
 - 0s - loss: -2.0196e+01 - acc: 0.9464
Epoch 915/

### Evaluate the Model

In [14]:
# Evaluate the model using the testing data
m_score_list = model.evaluate(X_test_scaled, y_test_scaled, verbose=2)
m_label_list = model.metrics_names

# Print the metrics and their associated labels
for i in range(len(m_score_list)):
    print(f"{m_label_list[i].title()}: {m_score_list[i]:0.4f}")

Loss: -20.0706
Acc: 0.9737


In [20]:
# Make some predictions using the testing data
s_min = 10
s_max = 20
y_encoded_predictions = model.predict_classes(X_test_scaled[s_min:s_max])
y_encoded_actual = [ np.argmax(y_val) for y_val in y_test[s_min:s_max] ]

# Get the labels from the one-hot encoded predictions and y_test values
# prediction_labels = label_encoder.inverse_transform(encoded_predictions)   # No Label Encoder needed this time
# Use the iris dataset target_names array to get the labels
y_prediction_labels = [ iris.target_names[i] for i in y_encoded_predictions ]
y_actual_labels = [ iris.target_names[i] for i in y_encoded_actual ]

predict_sample_df = pd.DataFrame(
    {'Actual': y_actual_labels,
     'Prediction': y_prediction_labels})

predict_sample_df['Correct'] = (predict_sample_df['Actual'] == predict_sample_df['Prediction'])
predict_sample_df

,Actual,Prediction,Correct
0,setosa,setosa,True
1,setosa,setosa,True
2,setosa,setosa,True
3,versicolor,versicolor,True
4,setosa,setosa,True
5,versicolor,versicolor,True
6,setosa,setosa,True
7,versicolor,versicolor,True
8,versicolor,versicolor,True
9,versicolor,versicolor,True


### Save the Fitted Model

In [21]:
# Save the fitted model
model.save("resources/basic_model.h5")

### Load a Fitted Model

In [ ]:
# Reload the fitted model
prev_model = load_model("resources/basic_model.h5")

In [23]:
# Make some predictions using the testing data
s_min = 10
s_max = 20
y_encoded_predictions = prev_model.predict_classes(X_test_scaled[s_min:s_max])
y_encoded_actual = [ np.argmax(y_val) for y_val in y_test[s_min:s_max] ]

# Get the labels from the one-hot encoded predictions and y_test values
# prediction_labels = label_encoder.inverse_transform(encoded_predictions)   # No Label Encoder needed this time
# Use the iris dataset target_names array to get the labels
y_prediction_labels = [ iris.target_names[i] for i in y_encoded_predictions ]
y_actual_labels = [ iris.target_names[i] for i in y_encoded_actual ]

# Create a DataFrame of actual vs. predicted results
predict_prev_model_df = pd.DataFrame(
    {'Actual': y_actual_labels,
     'Prediction': y_prediction_labels})

# Flag the predictions as being correct or not
predict_prev_model_df['Correct'] = (predict_prev_model_df['Actual'] == predict_prev_model_df['Prediction'])
predict_prev_model_df

,Actual,Prediction,Correct
0,setosa,setosa,True
1,setosa,setosa,True
2,setosa,setosa,True
3,versicolor,versicolor,True
4,setosa,setosa,True
5,versicolor,versicolor,True
6,setosa,setosa,True
7,versicolor,versicolor,True
8,versicolor,versicolor,True
9,versicolor,versicolor,True
